In [13]:
# This scrip is from the user 'Pollyanna' on Discord 1196170240606871643
from PIL import Image, ImageSequence, ImageChops
import os, os.path
import glob

# How many frames to duplicate down to. The number of clones in the final gif will be roughly the actual number of frames in the gif divided by this number
FinalFrameNumber = 2  # Increase this to reduce overlap and create smoother interpolation
# A minimum number of actual frames in the output accomplished by looping the final frames enough times.
MinimumOutputFrames = 15 # Increase this to have more frames in the final output
# Output Framerate (in milliseconds per frame)
MsPerFrame = 12  # Adjust this to control the speed of the animation

# Change this to the file you want to clone. Final result will be 'FinalPoseResult.gif' in the folder this script was run from
InputImage = Image.open("AnimateDiff_00011.gif")

# Are later frames added above or below previous frames?
LaterFramesAboveEarlierFrames = False

InputFrames = [f.copy().convert("RGB") for f in ImageSequence.Iterator(InputImage)]
NumInputFrames = len(InputFrames)

# Create an output image of the starting frames
FinishedProductFrames = []
for i in range(FinalFrameNumber):
	# Start with a base frame from the input pose animation
	currFrame = InputFrames[i]
	
	# Iterate through the rest of the frames, adding the non-black pixels to this base frame
	for j in range(i + FinalFrameNumber,NumInputFrames,FinalFrameNumber):
		newFrame = InputFrames[j]
		newPixelData = newFrame.getdata()
		currPixelData = currFrame.getdata()
		ind = 0
		newData = []
		for datum in newPixelData:
			# If the pixel isn't black, it's pose information, so copy it
			if datum != (0,0,0) and (currPixelData[ind] == (0,0,0) or LaterFramesAboveEarlierFrames) :
				newData.append(datum)
			else:
				newData.append(currPixelData[ind])
			ind = ind + 1
		currFrame.putdata(newData)
	FinishedProductFrames.append(currFrame)
	
OutputFrames = []
while(len(OutputFrames) < MinimumOutputFrames):
	for i in range(len(FinishedProductFrames)):
		OutputFrames.append(FinishedProductFrames[i])
	
# Save the animation
OutputFrames[0].save(
	'./FinalPoseResult.gif',
	save_all = True, 
	append_images = OutputFrames[0:],
	loop = 0,
	duration=MsPerFrame,
)